In [5]:
import pandas as pd
import numpy as np

# 載入數據
file_path = '新竹_2021.xls'
data = pd.read_excel(file_path, skiprows=[1])

# 將無效值轉換為 NaN，並將 'NR' 替換為 0 表示無降雨
data.replace({'#                              ': np.nan,
              '*                              ': np.nan,
              'X                              ': np.nan,
              'A                              ': np.nan,
              'NR                              ': 0,
              '': np.nan}, inplace=True)

# 提取日期資訊，並篩選 10 月、11 月和 12 月的數據
data['Date'] = pd.to_datetime(data["日期                  "], errors='coerce')
data_oct_nov = data[(data['Date'].dt.month == 10) | (data['Date'].dt.month == 11)]
data_dec = data[data['Date'].dt.month == 12]

def fill_missing_with_nearest_avg(data):
    for column in data.columns:
        if column not in ['Date', "日期                  ", "測站                  ", "測項                  "]:
            # 將該列轉換為數值型，無法轉換的值設為 NaN
            data[column] = pd.to_numeric(data[column], errors='coerce')
            
            # 遍歷每個數據行並填補 NaN 值
            for i in range(len(data)):
                if pd.isna(data.iloc[i][column]):
                    offset = 1
                    values_to_average = []
                    
                    # 只取前後一個小時的有效數據
                    while not values_to_average and (i - offset >= 0 or i + offset < len(data)):
                        # 向前找數值
                        if i - offset >= 0:
                            prev_value = data.iloc[i - offset][column]
                            if not pd.isna(prev_value):
                                values_to_average.append(prev_value)
                                
                        # 向後找數值
                        if i + offset < len(data):
                            next_value = data.iloc[i + offset][column]
                            if not pd.isna(next_value):
                                values_to_average.append(next_value)
                        
                        offset += 1
                    
                    # 如果找到有效數值，則計算平均值來填補 NaN
                    if values_to_average:
                        data.iloc[i, data.columns.get_loc(column)] = np.mean(values_to_average)
                        
    return data

# 填補缺失值
data_oct_nov = fill_missing_with_nearest_avg(data_oct_nov)
data_dec = fill_missing_with_nearest_avg(data_dec)

# 將 10 月和 11 月的數據設為訓練集，12 月的數據設為測試集
train_data = data_oct_nov
test_data = data_dec

C:\Users\User\AppData\Local\Temp\ipykernel_37476\159491926.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = pd.to_numeric(data[column], errors='coerce')


In [6]:
train_data.set_index('Date', inplace=True)
train_data

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,新竹,2021-10-01,AMB_TEMP,28.30,28.30,27.80,27.80,27.60,27.60,27.70,...,31.60,31.40,30.90,30.50,30.20,29.80,29.40,29.10,28.70,28.20
2021-10-01,新竹,2021-10-01,CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,...,1.96,1.97,2.01,2.06,2.07,2.05,2.04,2.03,2.08,2.08
2021-10-01,新竹,2021-10-01,CO,0.34,0.30,0.30,0.29,0.30,0.33,0.44,...,0.25,0.27,0.32,0.43,0.45,0.45,0.43,0.42,0.43,0.39
2021-10-01,新竹,2021-10-01,NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,...,0.04,0.06,0.05,0.17,0.24,0.22,0.16,0.14,0.16,0.14
2021-10-01,新竹,2021-10-01,NO,0.90,0.20,0.50,0.40,0.20,0.60,2.20,...,0.50,0.50,0.50,0.30,0.30,0.30,0.30,0.30,0.40,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-30,新竹,2021-11-30,THC,2.53,2.65,2.40,2.39,2.33,2.07,1.95,...,2.06,2.08,2.12,2.21,2.19,2.21,2.20,2.18,2.15,2.12
2021-11-30,新竹,2021-11-30,WD_HR,99.00,146.00,158.00,192.00,244.00,57.00,68.00,...,47.00,45.00,59.00,58.00,53.00,58.00,56.00,47.00,45.00,51.00
2021-11-30,新竹,2021-11-30,WIND_DIREC,128.00,249.00,150.00,211.00,248.00,53.00,17.00,...,55.00,55.00,60.00,59.00,52.00,57.00,54.00,35.00,56.00,52.00


In [9]:
test_data.set_index('Date', inplace=True)

In [123]:
train_data[0]

Date
2021-10-01     28.30
2021-10-01      2.04
2021-10-01      0.34
2021-10-01      0.17
2021-10-01      0.90
               ...  
2021-11-30      2.53
2021-11-30     99.00
2021-11-30    128.00
2021-11-30      0.30
2021-11-30      0.30
Name: 0, Length: 1098, dtype: float64

In [127]:
train_data[0][18]

C:\Users\User\AppData\Local\Temp\ipykernel_35684\3132136263.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_data[0][18]


27.8

In [10]:
data = [[] for _ in range(18)]

# 填充每一個欄位的數據
for k in range(18):
    for i in range(61):
        for j in range(24):
            data[k].append(train_data[j][k + i * 18])

# 將每個欄位的數據轉換為 DataFrame，並轉置以符合 18 欄和 1464 列的要求
df = pd.DataFrame(data).T

# 查看 DataFrame 結果
print(df)
train_data_flat = df
print("轉換後的資料形狀:",  train_data_flat.shape)
train_data_flat.head()

C:\Users\User\AppData\Local\Temp\ipykernel_17492\645424105.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[k].append(train_data[j][k + i * 18])


        0     1     2     3    4     5     6     7     8     9    10    11  \
0     28.3  2.04  0.34  0.17  0.9  18.8  19.8  16.0  28.0  28.0  0.0  71.0   
1     28.3  2.02  0.30  0.13  0.2  11.9  12.2  21.5  24.0  22.0  0.0  66.0   
2     27.8  2.12  0.30  0.12  0.5  15.1  15.6  16.9  29.0  26.0  0.0  76.0   
3     27.8  2.18  0.29  0.14  0.4  12.8  13.2  16.4  32.0  24.0  0.0  79.0   
4     27.6  2.19  0.30  0.17  0.2  14.9  15.1  12.6  31.0  28.0  0.0  81.0   
...    ...   ...   ...   ...  ...   ...   ...   ...   ...   ...  ...   ...   
1459  20.3  2.10  0.46  0.11  1.1  12.5  13.7  34.9  58.0  36.0  0.0  65.0   
1460  19.9  2.10  0.45  0.10  1.0  10.5  11.5  33.5  46.0  34.0  0.0  62.0   
1461  19.4  2.09  0.42  0.09  1.1   8.8  10.0  33.8  52.0  32.0  0.0  58.0   
1462  19.0  2.07  0.37  0.08  0.9   8.2   9.2  32.6  54.0  32.0  0.0  56.0   
1463  18.4  2.05  0.33  0.07  0.8   7.1   8.0  34.7  55.0  25.0  0.0  52.0   

       12    13     14     15   16   17  
0     1.8  2.21   62.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16.0,28.0,28.0,0.0,71.0,1.8,2.21,62.0,33.0,0.8,0.7
1,28.3,2.02,0.30,0.13,0.2,11.9,12.2,21.5,24.0,22.0,0.0,66.0,1.0,2.15,121.0,183.0,1.2,0.6
2,27.8,2.12,0.30,0.12,0.5,15.1,15.6,16.9,29.0,26.0,0.0,76.0,1.0,2.24,164.0,160.0,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32.0,24.0,0.0,79.0,1.3,2.32,156.0,151.0,0.6,0.4
4,27.6,2.19,0.30,0.17,0.2,14.9,15.1,12.6,31.0,28.0,0.0,81.0,1.5,2.36,110.0,90.0,0.9,0.5


In [11]:
data_2 = [[] for _ in range(18)]

# 填充每一個欄位的數據
for k in range(18):
    for i in range(31):
        for j in range(24):
            data_2[k].append(test_data[j][k + i * 18])

# 將每個欄位的數據轉換為 DataFrame，並轉置以符合 18 欄和 1464 列的要求
df_2 = pd.DataFrame(data_2).T

# 查看 DataFrame 結果
print(df_2)
test_data_flat = df_2
print("轉換後的資料形狀:",  test_data_flat.shape)
test_data_flat.head()

       0     1     2     3    4     5     6     7     8     9    10    11  \
0    17.8  2.04  0.30  0.05  0.8   6.2   7.0  33.2  47.0  22.0  0.0  49.0   
1    17.2  2.04  0.29  0.06  0.4   6.6   7.0  31.2  48.0  26.0  0.0  48.0   
2    16.7  2.04  0.28  0.04  0.4   6.0   6.4  30.8  47.0  24.0  0.0  47.0   
3    16.3  2.04  0.27  0.03  0.3   5.6   5.9  32.8  45.0  23.0  0.0  46.0   
4    15.9  2.03  0.26  0.05  0.3   5.7   6.1  33.0  42.0  17.0  0.0  44.0   
..    ...   ...   ...   ...  ...   ...   ...   ...   ...   ...  ...   ...   
739  15.8  2.01  0.29  0.09  0.7  11.6  12.3  32.1  19.0  13.0  0.0  71.0   
740  15.6  2.04  0.33  0.11  0.8  13.6  14.4  29.6  18.0  10.0  0.0  74.0   
741  15.7  2.03  0.32  0.10  0.7  13.6  14.4  30.3   8.0  11.0  0.0  78.0   
742  15.9  2.01  0.28  0.08  0.5  11.8  12.3  31.9  10.0   9.0  0.0  79.0   
743  16.1  2.00  0.25  0.05  0.4   9.2   9.7  33.6   7.0   7.0  0.0  78.0   

      12    13    14    15   16   17  
0    1.9  2.09  57.0  52.0  4.5  4.1

C:\Users\User\AppData\Local\Temp\ipykernel_17492\1175835615.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_2[k].append(test_data[j][k + i * 18])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,17.8,2.04,0.30,0.05,0.8,6.2,7.0,33.2,47.0,22.0,0.0,49.0,1.9,2.09,57.0,52.0,4.5,4.1
1,17.2,2.04,0.29,0.06,0.4,6.6,7.0,31.2,48.0,26.0,0.0,48.0,1.1,2.10,57.0,58.0,5.9,4.3
2,16.7,2.04,0.28,0.04,0.4,6.0,6.4,30.8,47.0,24.0,0.0,47.0,1.1,2.08,58.0,57.0,4.8,4.1
3,16.3,2.04,0.27,0.03,0.3,5.6,5.9,32.8,45.0,23.0,0.0,46.0,1.2,2.07,56.0,51.0,4.6,4.2
4,15.9,2.03,0.26,0.05,0.3,5.7,6.1,33.0,42.0,17.0,0.0,44.0,1.5,2.08,49.0,40.0,4.5,3.8


In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# 定義超參數範圍
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}



# 提取 PM2.5 和所有特徵
pm25_train = train_data_flat[9].values
all_data_train = train_data_flat.values
pm25_test = test_data_flat[9].values
all_data_test = test_data_flat.values

# 建立逐時資料特徵和目標
X_pm25_train_1hr = np.array([pm25_train[i:i+6] for i in range(len(pm25_train) - 6)])
y_train_1hr = pm25_train[6:]
X_pm25_train_6hr = np.array([pm25_train[i:i+6] for i in range(len(pm25_train) - 11)])
y_train_6hr = pm25_train[11:]

X_all_train_1hr = np.array([all_data_train[i:i+6].flatten() for i in range(len(pm25_train) - 6)])
X_all_train_6hr = np.array([all_data_train[i:i+6].flatten() for i in range(len(pm25_train) - 11)])

# 標準化訓練資料
scaler_pm25 = StandardScaler()
scaler_all = StandardScaler()
X_pm25_train_1hr = scaler_pm25.fit_transform(X_pm25_train_1hr)
X_pm25_train_6hr = scaler_pm25.transform(X_pm25_train_6hr)
X_all_train_1hr = scaler_all.fit_transform(X_all_train_1hr)
X_all_train_6hr = scaler_all.transform(X_all_train_6hr)

# 準備12月測試集資料
X_pm25_test_1hr = np.array([pm25_test[i:i+6] for i in range(len(pm25_test) - 6)])
y_test_1hr = pm25_test[6:]
X_pm25_test_6hr = np.array([pm25_test[i:i+6] for i in range(len(pm25_test) - 11)])
y_test_6hr = pm25_test[11:]

X_all_test_1hr = np.array([all_data_test[i:i+6].flatten() for i in range(len(pm25_test) - 6)])
X_all_test_6hr = np.array([all_data_test[i:i+6].flatten() for i in range(len(pm25_test) - 11)])

# 使用訓練的標準化器標準化測試資料
X_pm25_test_1hr = scaler_pm25.transform(X_pm25_test_1hr)
X_pm25_test_6hr = scaler_pm25.transform(X_pm25_test_6hr)
X_all_test_1hr = scaler_all.transform(X_all_test_1hr)
X_all_test_6hr = scaler_all.transform(X_all_test_6hr)



# 線性回歸模型
lr_model = LinearRegression()
lr_model.fit(X_all_train_1hr, y_train_1hr)
y_pred_1hr = lr_model.predict(X_all_test_1hr)
mae_1hr = mean_absolute_error(y_test_1hr, y_pred_1hr)
print(f"Linear Regression - MAE for 1hr target: {mae_1hr}")

lr_model.fit(X_all_train_6hr, y_train_6hr)
y_pred_6hr = lr_model.predict(X_all_test_6hr)
mae_6hr = mean_absolute_error(y_test_6hr, y_pred_6hr)
print(f"Linear Regression - MAE for 6hr target: {mae_6hr}")


# 線性回歸模型
lr_model = LinearRegression()
lr_model.fit(X_pm25_train_1hr, y_train_1hr)
y_pred_1hr = lr_model.predict(X_pm25_test_1hr)
mae_1hr = mean_absolute_error(y_test_1hr, y_pred_1hr)
print(f"Linear Regression - MAE for 1hr target (using only PM2.5): {mae_1hr}")

lr_model.fit(X_pm25_train_6hr, y_train_6hr)
y_pred_6hr = lr_model.predict(X_pm25_test_6hr)
mae_6hr = mean_absolute_error(y_test_6hr, y_pred_6hr)
print(f"Linear Regression - MAE for 6hr target (using only PM2.5): {mae_6hr}")


# 設定 XGBoost 的超參數範圍
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# 進行超參數調整
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=20,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# 使用所有特徵的 1hr 目標進行超參數調整
random_search.fit(X_all_train_1hr, y_train_1hr)
print("Best parameters for all features, 1hr target: ", random_search.best_params_)
best_xgb_model = random_search.best_estimator_

# 1hr 預測結果
y_pred_1hr_best = best_xgb_model.predict(X_all_test_1hr)
mae_1hr_best = mean_absolute_error(y_test_1hr, y_pred_1hr_best)
print(f"Optimized XGBoost - MAE for 1hr target (all features): {mae_1hr_best}")

# 使用最佳參數在 6hr 目標進行訓練和測試
best_xgb_model.fit(X_all_train_6hr, y_train_6hr)
y_pred_6hr_best = best_xgb_model.predict(X_all_test_6hr)
mae_6hr_best = mean_absolute_error(y_test_6hr, y_pred_6hr_best)
print(f"Optimized XGBoost - MAE for 6hr target (all features): {mae_6hr_best}")


# 使用最佳超參數的 XGBoost 模型，僅使用 PM2.5
xgb_model_pm25 = xgb.XGBRegressor(**random_search.best_params_, objective='reg:squarederror', random_state=42)
xgb_model_pm25.fit(X_pm25_train_1hr, y_train_1hr)
y_pred_1hr_xgb_pm25 = xgb_model_pm25.predict(X_pm25_test_1hr)
mae_1hr_xgb_pm25 = mean_absolute_error(y_test_1hr, y_pred_1hr_xgb_pm25)
print(f"Optimized XGBoost - MAE for 1hr target (using only PM2.5): {mae_1hr_xgb_pm25}")

xgb_model_pm25.fit(X_pm25_train_6hr, y_train_6hr)
y_pred_6hr_xgb_pm25 = xgb_model_pm25.predict(X_pm25_test_6hr)
mae_6hr_xgb_pm25 = mean_absolute_error(y_test_6hr, y_pred_6hr_xgb_pm25)
print(f"Optimized XGBoost - MAE for 6hr target (using only PM2.5): {mae_6hr_xgb_pm25}")


Linear Regression - MAE for 1hr target: 2.8967475782724033
Linear Regression - MAE for 6hr target: 4.380562664954107
Linear Regression - MAE for 1hr target (using only PM2.5): 2.8314570956670893
Linear Regression - MAE for 6hr target (using only PM2.5): 4.408783263479132
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters for all features, 1hr target:  {'subsample': 0.6, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
Optimized XGBoost - MAE for 1hr target (all features): 3.003468539333602
Optimized XGBoost - MAE for 6hr target (all features): 4.6250029364842185
Optimized XGBoost - MAE for 1hr target (using only PM2.5): 2.9640485946401993
Optimized XGBoost - MAE for 6hr target (using only PM2.5): 4.515008888660967
